In [52]:
import torchviz
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

In [76]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        #
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        
        #
        self.auxMPool = nn.MaxPool1d(kernel_size = 10, stride = 10)
        self.avgpool = nn.AdaptiveAvgPool1d(12)
        
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        #
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 3840, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        # Feature Learning Head
        
        x = torch.moveaxis(x,1,0)
        x = torch.tensor(x, dtype=torch.float32)

        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
#         x_aux = x_aux.view(-1,3840)
        
        #Classification Head
#         x_aux = F.relu(self.fc1((x_aux)))
#         x_aux = self.Dropout(x_aux)
#         x_aux = F.relu(self.fc2(x_aux))
#         x_aux = self.fc3(x_aux)
        
        
#         print(x_S.shape)
#         print(x_M.shape)
#         print(x_L.shape)
#         print(x_O.shape)
        
        #
        x = torch.cat((x_S,x_M,x_L,x_O),0)
        
        print(x.shape)
        
        x_aux = self.avgpool(x)
        #Classification Head
        x_aux = x_aux.view(-1,3840)
        x_aux = F.relu(self.fc1((x_aux)))
        x_aux = self.Dropout(x_aux)
        x_aux = F.relu(self.fc2(x_aux))
        x_aux = self.fc3(x_aux)
#         x_aux = F.softmax(x_aux)

        x = self.MPool2(F.relu(self.conv2(x)))
        print(x.shape)
        
        # Flattening
        x = x.view(-1,3840)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = F.softmax(x)
        
        return (x, x_aux)



In [77]:
num_filters = 64
learning_rate = 0.001
num_epochs = 200
use_cuda = True
early_stop = 3
batch_size = 32
kernels = [1, 3, 5]
min_delta = -0.025
min_epochs = 1
num_workers = 4
small_kernel, medium_kernel, large_kernel = kernels

In [78]:
one_CNN = MSCB(small_kernel, medium_kernel, large_kernel, num_filters)

In [79]:
x = torch.randn(56, 100)


In [80]:
from torchviz import make_dot

make_dot(one_CNN, params=dict(list(one_CNN.named_parameters()))).render("cnn_torchviz", format="png")

AttributeError: 'MSCB' object has no attribute 'size'

In [81]:
one_CNN.named_parameters()

<generator object Module.named_parameters at 0x0000024B2C594B30>

In [85]:
x = torch.randn(1500,56).requires_grad_(True)
y = one_CNN(x)

from torchviz import make_dot
make_dot(one_CNN, params=dict(list(one_CNN.named_parameters()))).render("torchviz", format="png")

torch.Size([320, 300])
torch.Size([64, 60])


C:\Users\GillA\AppData\Local\Temp\ipykernel_6308\922151754.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


AttributeError: 'MSCB' object has no attribute 'size'

In [83]:
320*4

1280